In [71]:
import os
import json
import requests
import base64
import pandas as pd
import pickle

In [2]:
def call_jquants_api(params: dict, idtoken: str, apitype: str, code: str = None):
    """
    J-QuantsのAPIを試すメソッド。

    Parameters
    ----------
    params : dict
        リクエストパラメータ。
    idtoken : str
        idTokenはログイン後の画面からご確認いただけます。
    apitype: str
        APIの種類。"news", "prices", "lists"などがあります。
    code: str
        銘柄を指定するAPIの場合に設定します。

    Returns
    -------
    resjson : dict
        APIレスポンス(json形式)
    """
    datefrom = params.get("datefrom", None)
    dateto = params.get("dateto", None)
    date = params.get("date", None)
    includedetails = params.get("includedetails", "false")
    keyword = params.get("keyword", None)
    headline = params.get("headline", None)
    paramcode = params.get("code", None)
    nexttoken = params.get("nextToken", None)
    headers = {"accept": "application/json", "Authorization": idtoken}
    data = {
        "from": datefrom,
        "to": dateto,
        "includeDetails": includedetails,
        "nextToken": nexttoken,
        "date": date,
        "keyword": keyword,
        "headline": headline,
        "code": paramcode,
    }

    if code:
        code = "/" + code
        r = requests.get(
            "https://api.jpx-jquants.com/" + apitype + code,
            params=data,
            headers=headers,
        )
    else:
        r = requests.get(
            "https://api.jpx-jquants.com/" + apitype, params=data, headers=headers
        )
    resjson = json.loads(r.text)
    return resjson

In [63]:
# 使用するモジュールのインポート
from datetime import datetime as dt
from datetime import timedelta

# 日付条件の設定
strdt = dt.strptime("2020-12-01", '%Y-%m-%d')  # 開始日
enddt = dt.strptime("2021-03-27", '%Y-%m-%d')  # 終了日
#strdt = dt.strptime("2021-01-01", '%Y-%m-%d')  # 開始日
#enddt = dt.strptime("2021-01-05", '%Y-%m-%d')  # 終了日

# 日付差の日数を算出（リストに最終日も含めたいので、＋１しています）
days_num = (enddt - strdt).days + 1  # （参考）括弧の部分はtimedelta型のオブジェクトになります

In [64]:
# シンプルにforとappendを使用した場合
datelist = []
for i in range(days_num):
    datelist.append(strdt + timedelta(days=i))

# Stock Lists

In [89]:
idtk = "eyJraWQiOiJTelhkZGxsSEQrR0dyS0ZKRmdZZkNKdUFZdmxIOFdrVks5bXRcLzgrdFduTT0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiI1YWYwN2U3YS1lMThkLTRhZTItYmRkZS0wMzllMzhjYTI2ZWEiLCJhdWQiOiI0bHYzZ2RuOXA1ZnF0cDcydmMzY3I5cG5xbSIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJldmVudF9pZCI6IjU4ZmY4ZmUxLWI0MjAtNDUxOS04ZDNhLWU4MDI2Yzk3ZmU0ZCIsInRva2VuX3VzZSI6ImlkIiwiYXV0aF90aW1lIjoxNjE2MjE2Mzc4LCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAuYXAtbm9ydGhlYXN0LTEuYW1hem9uYXdzLmNvbVwvYXAtbm9ydGhlYXN0LTFfM2o5eHZhYXlmIiwibmlja25hbWUiOiJTZWlzdWtlIFRhYmF5YXNoaSIsImNvZ25pdG86dXNlcm5hbWUiOiI1YWYwN2U3YS1lMThkLTRhZTItYmRkZS0wMzllMzhjYTI2ZWEiLCJleHAiOjE2MTY4NTc3NzMsImlhdCI6MTYxNjg1NDE3MywiZW1haWwiOiJzdG1vZGUyMDAzQGdtYWlsLmNvbSJ9.KndH69MJOFW0kWWzl6NsPb_4c1xCLfqfPoytDbLqfvP32NJwDutBzQrNfKRdgIT1wxDiPskPJf8_Ydtl3XWnU3lylLsnbQs2TupJBGP7tGoYJL4fXSYIi_G2K2xuAcKg8bUEYxE2xguGUVJ8N76SGCwOQZTbhWw9zvCWYy0IHMEWD1n5F5HHEeddj5uM3NB2VVsY5Qrmb695siWbLc4oQQAJccmF0ApmwET7G8oJ-Hko-XYtMDe3g--XMyurIl4GKe6sAZo2oCYHeFnYaJNSK5Od09jeOq04CxMBNbeM9JY38tYQ7cNpzX-SmjLATFDUpudwgGREnlanbeMHRkF6SA"

In [90]:
paramdict = {}
paramdict["includedetails"] = "True"
responce = call_jquants_api(paramdict, idtk, "lists")

In [101]:
lists = pd.DataFrame()

is_next = True
tmp_responce  = pd.DataFrame()

paramdict = {}
paramdict["includedetails"] = "True"

while is_next:
    responce = call_jquants_api(paramdict, idtk, "lists")
    if responce['list'] == []:
        is_next = False
    else:
        tmp_responce = pd.DataFrame(responce['list'])
        lists = lists.append(tmp_responce)
        try:
            paramdict['nextToken'] = responce['nextToken']
        except KeyError:
            is_next =  False


In [103]:
model_path = os.path.join(os.path.dirname("__file__"), "../../get_data")

# tag::save_model[]
# モデル保存先ディレクトリを作成
os.makedirs(model_path, exist_ok=True)
with open(os.path.join(model_path, "stock_lists.pkl"), "wb") as f:
    # モデルをpickle形式で保存
    pickle.dump(lists, f)

# Prices

In [65]:
prices = pd.DataFrame()

for date in datelist:
    print(date.strftime("%Y-%m-%d"))
    is_next = True
    tmp_responce  = pd.DataFrame()
    
    paramdict = {}
    paramdict["date"] = date.strftime("%Y-%m-%d")
    paramdict["includedetails"] = "True"
                
    while is_next:
        responce = call_jquants_api(paramdict, idtk, "prices")
        if responce['prices'] == []:
            is_next = False
        else:
            tmp_responce = pd.DataFrame(responce['prices'])
            prices = prices.append(tmp_responce)
            try:
                paramdict['nextToken'] = responce['nextToken']
            except KeyError:
                is_next =  False


2020-12-01
2020-12-02
2020-12-03
2020-12-04
2020-12-05
2020-12-06
2020-12-07
2020-12-08
2020-12-09
2020-12-10
2020-12-11
2020-12-12
2020-12-13
2020-12-14
2020-12-15
2020-12-16
2020-12-17
2020-12-18
2020-12-19
2020-12-20
2020-12-21
2020-12-22
2020-12-23
2020-12-24
2020-12-25
2020-12-26
2020-12-27
2020-12-28
2020-12-29
2020-12-30
2020-12-31
2021-01-01
2021-01-02
2021-01-03
2021-01-04
2021-01-05
2021-01-06
2021-01-07
2021-01-08
2021-01-09
2021-01-10
2021-01-11
2021-01-12
2021-01-13
2021-01-14
2021-01-15
2021-01-16
2021-01-17
2021-01-18
2021-01-19
2021-01-20
2021-01-21
2021-01-22
2021-01-23
2021-01-24
2021-01-25
2021-01-26
2021-01-27
2021-01-28
2021-01-29
2021-01-30
2021-01-31
2021-02-01
2021-02-02
2021-02-03
2021-02-04
2021-02-05
2021-02-06
2021-02-07
2021-02-08
2021-02-09
2021-02-10
2021-02-11
2021-02-12
2021-02-13
2021-02-14
2021-02-15
2021-02-16
2021-02-17
2021-02-18
2021-02-19
2021-02-20
2021-02-21
2021-02-22
2021-02-23
2021-02-24
2021-02-25
2021-02-26
2021-02-27
2021-02-28
2021-03-01

In [73]:
model_path = os.path.join(os.path.dirname("__file__"), "../../get_data")

# tag::save_model[]
# モデル保存先ディレクトリを作成
os.makedirs(model_path, exist_ok=True)
with open(os.path.join(model_path, "stock_prices.pkl"), "wb") as f:
    # モデルをpickle形式で保存
    pickle.dump(prices, f)

# Stock Fins

In [82]:
fins = pd.DataFrame()

for date in datelist:
    print(date.strftime("%Y-%m-%d"))
    is_next = True
    tmp_responce  = pd.DataFrame()
    
    paramdict = {}
    paramdict["date"] = date.strftime("%Y-%m-%d")
    paramdict["includedetails"] = "True"
                
    while is_next:
        responce = call_jquants_api(paramdict, idtk, "stockfins")
        if responce['stockfin'] == []:
            is_next = False
        else:
            tmp_responce = pd.DataFrame(responce['stockfin'])
            fins = fins.append(tmp_responce)
            try:
                paramdict['nextToken'] = responce['nextToken']
            except KeyError:
                is_next =  False


2020-12-01
2020-12-02
2020-12-03
2020-12-04
2020-12-05
2020-12-06
2020-12-07
2020-12-08
2020-12-09
2020-12-10
2020-12-11
2020-12-12
2020-12-13
2020-12-14
2020-12-15
2020-12-16
2020-12-17
2020-12-18
2020-12-19
2020-12-20
2020-12-21
2020-12-22
2020-12-23
2020-12-24
2020-12-25
2020-12-26
2020-12-27
2020-12-28
2020-12-29
2020-12-30
2020-12-31
2021-01-01
2021-01-02
2021-01-03
2021-01-04
2021-01-05
2021-01-06
2021-01-07
2021-01-08
2021-01-09
2021-01-10
2021-01-11
2021-01-12
2021-01-13
2021-01-14
2021-01-15
2021-01-16
2021-01-17
2021-01-18
2021-01-19
2021-01-20
2021-01-21
2021-01-22
2021-01-23
2021-01-24
2021-01-25
2021-01-26
2021-01-27
2021-01-28
2021-01-29
2021-01-30
2021-01-31
2021-02-01
2021-02-02
2021-02-03
2021-02-04
2021-02-05
2021-02-06
2021-02-07
2021-02-08
2021-02-09
2021-02-10
2021-02-11
2021-02-12
2021-02-13
2021-02-14
2021-02-15
2021-02-16
2021-02-17
2021-02-18
2021-02-19
2021-02-20
2021-02-21
2021-02-22
2021-02-23
2021-02-24
2021-02-25
2021-02-26
2021-02-27
2021-02-28
2021-03-01

In [84]:
model_path = os.path.join(os.path.dirname("__file__"), "../../get_data")

# tag::save_model[]
# モデル保存先ディレクトリを作成
os.makedirs(model_path, exist_ok=True)
with open(os.path.join(model_path, "stock_fins.pkl"), "wb") as f:
    # モデルをpickle形式で保存
    pickle.dump(fins, f)

# Stock Labels

In [85]:
paramdict = {}
paramdict["date"] = "2018-05-31"
paramdict["includedetails"] = "true"
responce = call_jquants_api(paramdict, idtk, "stocklabels")

In [87]:
labels = pd.DataFrame()

for date in datelist:
    print(date.strftime("%Y-%m-%d"))
    is_next = True
    tmp_responce  = pd.DataFrame()
    
    paramdict = {}
    paramdict["date"] = date.strftime("%Y-%m-%d")
    paramdict["includedetails"] = "True"
                
    while is_next:
        responce = call_jquants_api(paramdict, idtk, "stocklabels")
        if responce['labels'] == []:
            is_next = False
        else:
            tmp_responce = pd.DataFrame(responce['labels'])
            labels = labels.append(tmp_responce)
            try:
                paramdict['nextToken'] = responce['nextToken']
            except KeyError:
                is_next =  False

2020-12-01
2020-12-02
2020-12-03
2020-12-04
2020-12-05
2020-12-06
2020-12-07
2020-12-08
2020-12-09
2020-12-10
2020-12-11
2020-12-12
2020-12-13
2020-12-14
2020-12-15
2020-12-16
2020-12-17
2020-12-18
2020-12-19
2020-12-20
2020-12-21
2020-12-22
2020-12-23
2020-12-24
2020-12-25
2020-12-26
2020-12-27
2020-12-28
2020-12-29
2020-12-30
2020-12-31
2021-01-01
2021-01-02
2021-01-03
2021-01-04
2021-01-05
2021-01-06
2021-01-07
2021-01-08
2021-01-09
2021-01-10
2021-01-11
2021-01-12
2021-01-13
2021-01-14
2021-01-15
2021-01-16
2021-01-17
2021-01-18
2021-01-19
2021-01-20
2021-01-21
2021-01-22
2021-01-23
2021-01-24
2021-01-25
2021-01-26
2021-01-27
2021-01-28
2021-01-29
2021-01-30
2021-01-31
2021-02-01
2021-02-02
2021-02-03
2021-02-04
2021-02-05
2021-02-06
2021-02-07
2021-02-08
2021-02-09
2021-02-10
2021-02-11
2021-02-12
2021-02-13
2021-02-14
2021-02-15
2021-02-16
2021-02-17
2021-02-18
2021-02-19
2021-02-20
2021-02-21
2021-02-22
2021-02-23
2021-02-24
2021-02-25
2021-02-26
2021-02-27
2021-02-28
2021-03-01

In [88]:
model_path = os.path.join(os.path.dirname("__file__"), "../../get_data")

# tag::save_model[]
# モデル保存先ディレクトリを作成
os.makedirs(model_path, exist_ok=True)
with open(os.path.join(model_path, "stock_labels.pkl"), "wb") as f:
    # モデルをpickle形式で保存
    pickle.dump(labels, f)